In [1]:
import pandas as pd
import math

In [34]:
match_history  = pd.read_csv('data/matches.csv',sep = ';')
match_history.drop("Unnamed: 0",axis=1,inplace=True)

In [37]:
match_history.head()

,date,team1,team2,map,event,event_type,event_tier,s1,s2
0,22/12/19,Vitality,MOUZ,Nuke,EPICENTER 2019,BigEvents,S,16,9
1,22/12/19,Vitality,MOUZ,Mirage,EPICENTER 2019,BigEvents,S,16,12
2,22/12/19,MOUZ,Vitality,Inferno,EPICENTER 2019,BigEvents,S,19,16
3,22/12/19,MOUZ,Evil Geniuses,Overpass,EPICENTER 2019,BigEvents,S,16,14
4,22/12/19,MOUZ,Evil Geniuses,Train,EPICENTER 2019,BigEvents,S,16,12


## Clean data

In [36]:
match_history.rename(columns={'Date':'date', 'Team1':"team1", 'Team2':"team2", 'Map':"map", 'Event':"event"},inplace=True)
match_history[['team1','s1']] = match_history['team1'].str.split('(',1,expand=True)
match_history[['team2','s2']] = match_history['team2'].str.split('(',1,expand=True)
match_history['team1'] = match_history['team1'].str.strip()
match_history['team2'] = match_history['team2'].str.strip()
match_history['s1'] = match_history['s1'].str[:-1]
match_history['s2'] = match_history['s2'].str[:-1]
match_history['map'] = match_history['map'].str[4:]

## Change type

### Score

In [41]:
match_history['s1'] = pd.to_numeric(match_history['s1'])
match_history['s2'] = pd.to_numeric(match_history['s2'])

### date

In [27]:
match_history['date'] = pd.to_datetime(match_history['date'])


In [28]:
match_history.dtypes


date          datetime64[ns]
team1                 object
team2                 object
map                   object
event                 object
event_type            object
event_tier            object
s1                    object
s2                    object
dtype: object

K-factor  	Used for players with ratings ...
k=40 for a player new to the rating list until the completion of events with a total of 30 games, and for all players until their 18th birthday, as long as their rating remains under 2300.
k=20 	for players who have always been rated under 2400.
k=10for players with any published rating of at least 2400 and at least 30 games played in previous events. Thereafter it remains permanently at 10.

Score after mathces,series or lan torunaments(greater difference for tournaments?? Performance Rating by tournament?)

elo starting at what?

elo for players?

In [15]:
def Probwin(rating1,rating2):
    # Function to calculate the Probability
    return 1.0 * 1.0 / (1 + 1.0 * math.pow(10, 1.0 * (rating1 - rating2) / 400))

def EloRating(Ra,Rb,k,d):
    # Function to calculate Elo rating
    # K is a constant.
    # d determines whether Player A wins or Player B.
    # To calculate the Winning
    # Probability of Player B
    Pb = Probwin(Ra, Rb)
 
    # To calculate the Winning
    # Probability of Player A
    Pa = Probwin(Rb, Ra)
      # Case -1 When Player A wins
    # Updating the Elo Ratings
    if (d == 1) :
        Ra = Ra + k * (1 - Pa)
        Rb = Rb + k * (0 - Pb)
    # Case -2 When Player B wins
    # Updating the Elo Ratings
    else :
        Ra = Ra + k * (0 - Pa)
        Rb = Rb + k * (1 - Pb)
    print("Updated Ratings:-")
    print("Ra =", round(Ra, 1)," Rb =", round(Rb, 1))
    return Ra,Rb

In [16]:
Ra = 1200
Rb = 1000
K = 30
d = 1
EloRating(Ra, Rb, K, d)

Updated Ratings:-
Ra = 1207.2  Rb = 992.8


(1207.2075922005613, 992.7924077994387)

In [40]:
match_history.sort_values(by=["Date"]).head(100)

,Date,Team1,Team2,Map,Event,event_type,event_tier
316,1/10/19,Vitality (11),ENCE (16),nuke Nuke,DreamHack Masters Malmö 2019,BigEvents,S
330,1/10/19,Envy (7),Astralis (16),trn Train,DreamHack Masters Malmö 2019,BigEvents,S
321,1/10/19,Astralis (16),G2 (7),d2 Dust2,DreamHack Masters Malmö 2019,BigEvents,S
312,1/10/19,fnatic (16),TYLOO (10),d2 Dust2,DreamHack Masters Malmö 2019,BigEvents,S
320,1/10/19,Envy (8),FURIA (16),vtg Vertigo,DreamHack Masters Malmö 2019,BigEvents,S
...,...,...,...,...,...,...,...
140,10/11/19,Astralis (16),100 Thieves (3),trn Train,IEM Beijing 2019,BigEvents,S
698,10/5/19,ENCE (16),Natus Vincere (4),mrg Mirage,BLAST Pro Series Madrid 2019,BigEvents,S
697,10/5/19,Astralis (16),Ninjas in Pyjamas (8),d2 Dust2,BLAST Pro Series Madrid 2019,BigEvents,S
696,10/5/19,Cloud9 (16),Giants (14),d2 Dust2,BLAST Pro Series Madrid 2019,BigEvents,S
